In [1]:
import pandas as pd
import altair as alt
alt.data_transformers.disable_max_rows() # Disable 5_000 rows limit
pd.set_option('display.max_colwidth', 1000) # Expand maximum column text display

In [2]:
findings_data = pd.read_csv("https://raw.githubusercontent.com/code-423n4/code423n4.com/main/_data/findings/findings.csv") # Set path accordingly
findings_data["contestid"] = findings_data["contest"]
contests_data = pd.read_csv("https://raw.githubusercontent.com/code-423n4/code423n4.com/main/_data/contests/contests.csv")
submissions_data = pd.read_csv("https://raw.githubusercontent.com/Krow10/code4rena-scraper/master/github_code4rena.csv")

In [3]:
df = pd.merge(submissions_data, submissions_data["tags"].str.get_dummies(';'), how='outer', left_index=True, right_index=True)

In [4]:
list(df.columns)

['contest',
 'contest_sponsor',
 'date',
 'handle',
 'address',
 'risk',
 'title',
 'issueId',
 'issueUrl',
 'tags',
 'issueCreation',
 '0 (Non-critical)',
 '1 (Low Risk)',
 '2 (Med Risk)',
 '3 (High Risk)',
 'ATokenYieldSource',
 'Acknowledged',
 'BadgerYieldSource',
 'BasePool',
 'BasePoolV2',
 'Confirmed',
 'ControlledToken',
 'Converter',
 'Disputed',
 'ERC20Rewards',
 'EmergencyBrake',
 'FYTokenFactory',
 'G (Gas Optimization)',
 'G-01',
 'G-02',
 'G-03',
 'G-04',
 'G-05',
 'G-06',
 'G-07',
 'G-08',
 'G-09',
 'G-10',
 'G-11',
 'G-12',
 'G-13',
 'G-14',
 'G-15',
 'G-16',
 'G-17',
 'G-18',
 'G-19',
 'G-20',
 'G-21',
 'G-22',
 'G-23',
 'G-24',
 'G-25',
 'G-26',
 'G-27',
 'G-28',
 'G-29',
 'Gas Report - High quality report',
 'GasThrottle',
 'GovernorAlpha',
 'H-01',
 'IdleYieldSource',
 'Index',
 'LPToken',
 'LPWrapper',
 'LinearVesting',
 'LiquidityBasedTWAP',
 'M-01',
 'M-02',
 'M-03',
 'M-04',
 'MitigationStarted',
 'Needs input from sponsor',
 'Notional',
 'Oracles',
 'PrizePool'

In [5]:
df2 = df[["handle", "issueId"]].copy()
df2 = pd.merge(df[(df.invalid == 1) & (df.contest != 999999)].groupby(["handle"])["invalid"].count(), df2, on="handle").drop_duplicates()
df2 = pd.merge(df2.groupby(["handle"])["issueId"].count(), df2, on="handle").drop_duplicates()
df2 = df2.rename(columns={'issueId_x': 'total', 'issueId_y': 'issueId'})
df2 = df2.drop('issueId', axis=1).drop_duplicates()
df2

,handle,total,invalid
0,0bi,5,4
5,0v3rf10w,98,38
103,0x040,19,3
122,0x0x0x,151,41
273,0x1337,28,13
...,...,...,...
20049,zhyd1997,3,3
20052,zishansami,8,2
20060,zmj,5,3
20065,zuhaibmohd,6,1


In [6]:
warden_dropdown = alt.binding_select(options=list(df2["handle"].unique()), name='Choose a warden:')
warden_selector = alt.selection_single(
    fields=['handle'], 
    bind=warden_dropdown, 
    name="warden_selector"
)

In [7]:
invalid = alt.Chart(df2).transform_fold(
    ["total", "invalid"]
).mark_bar(
    opacity=0.9
).encode(
    x=alt.X('handle:N', sort='-y', axis=alt.Axis(title='Wardens', grid=False, labelAngle=-45)),
    y=alt.Y('value:Q', title=""),
    color=alt.Color('key:N', scale=alt.Scale(scheme='set1')),
    order=alt.Order('key:N', sort='ascending')
)

In [8]:
invalid_normalize = alt.Chart(df2).transform_fold(
    ["total", "invalid"]
).mark_bar(
    opacity=0.9
).encode(
    x=alt.X('handle:N', sort='-y', axis=alt.Axis(title='Wardens', grid=False, labelAngle=-45)),
    y=alt.Y('value:Q', axis=alt.Axis(format='%'), stack="normalize"),
    color=alt.Color('key:N', scale=alt.Scale(scheme='set1')),
    order=alt.Order('key:N', sort='ascending')
)

In [9]:
mean_invalid_rule_line = alt.Chart(df2).transform_calculate(
    invalid_ratio="datum.invalid / datum.total"
).mark_rule(
    color='red',
    size=2,
    opacity=0.8,
).encode(
    y=alt.Y('mean(invalid_ratio):Q', title="")
)

## Invalid submissions stats
The first graph shows the total number of submissions (in blue) with the number of *invalid* submissions (in red) for each warden. Keep in mind the two number are stacked so the top of the bar indicates the **total** + **invalid** submissions.

The second graph is a normalized version of the first graph showing the ratio of invalid submissions for each warden with the red line being the average (~40%, exact number below the chart).

Select a specific warden using the dropdown. To return to the overall view, re-run the cell.

In [10]:
(invalid & (invalid_normalize + mean_invalid_rule_line)).add_selection(
    warden_selector
).transform_filter(
    warden_selector
).properties(
    title='Invalid submissions'
)

/opt/hostedtoolcache/Python/3.11.1/x64/lib/python3.11/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)

In [11]:
df2['invalid_ratio'] = df2.invalid / df2.total
'Average invalid submission ratio: ' + str(round(df2['invalid_ratio'].mean() * 100, 2)) + '%'

'Average invalid submission ratio: 45.95%'

# - - - - - - - - -

In [12]:
df3 = df[["handle", "issueId"]].copy()
df3 = pd.merge(df3, df[df['disagree with severity'] == 1].groupby('handle')["issueId"].count().add(df[df['disagree-with-severity'] == 1].groupby('handle')["issueId"].count(), fill_value=0), on="handle")
df3 = df3.rename(columns={'issueId_x': 'issueId', 'issueId_y': 'downgrade'})
df3 = df3.drop('issueId', axis=1).drop_duplicates()
df3 = pd.merge(df3, df2[['handle', 'total']], on="handle")
df3

,handle,downgrade,total
0,MaratCerby,8.0,60
1,BowTiedWardens,14.0,103
2,0x1f8b,36.0,188
3,GimelSec,29.0,142
4,cccz,42.0,157
...,...,...,...
283,technicallyty,1.0,3
284,polymorphism,1.0,4
285,Soul22,1.0,3
286,apajaresaguilera,1.0,2


In [13]:
downgrade = alt.Chart(df3).transform_fold(
    ["total", "downgrade"]
).mark_bar(
    opacity=0.9
).encode(
    x=alt.X('handle:N', sort='-y', axis=alt.Axis(title='Wardens', grid=False, labelAngle=-45)),
    y=alt.Y('value:Q', title=""),
    color=alt.Color('key:N', scale=alt.Scale(scheme='set1')),
    order=alt.Order('key:N', sort='ascending')
)

In [14]:
downgrade_normalize = alt.Chart(df3).transform_fold(
    ["total", "downgrade"]
).mark_bar(
    opacity=0.9
).encode(
    x=alt.X('handle:N', sort='-y', axis=alt.Axis(title='Wardens', grid=False, labelAngle=-45)),
    y=alt.Y('value:Q', axis=alt.Axis(format='%'), stack="normalize"),
    color=alt.Color('key:N', scale=alt.Scale(scheme='set1')),
    order=alt.Order('key:N', sort='ascending')
)

In [15]:
mean_downgrade_rule_line = alt.Chart(df3).transform_calculate(
    downgrade_ratio="datum.downgrade / datum.total"
).mark_rule(
    color='red',
    size=2,
    opacity=0.8,
).encode(
    y=alt.Y('mean(downgrade_ratio):Q', title="")
)

## Downgraded submissions stats
The first graph shows the total number of submissions (in blue) with the number of *downgraded* submissions (in red) for each warden. Keep in mind the two number are stacked so the top of the bar indicates the **total** + **downgraded** submissions.

The second graph is a normalized version of the first graph showing the ratio of downgraded submissions for each warden with the red line being the average (~15%, exact number below the chart).

Select a specific warden using the dropdown. To return to the overall view, re-run the cell.

In [16]:
(downgrade & (downgrade_normalize + mean_downgrade_rule_line)).add_selection(
    warden_selector
).transform_filter(
    warden_selector
).properties(
    title='Downgraded submissions'
)

/opt/hostedtoolcache/Python/3.11.1/x64/lib/python3.11/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)

In [17]:
df3['downgrade_ratio'] = df3.downgrade / df3.total
'Average downgraded submission ratio: ' + str(round(df3['downgrade_ratio'].mean() * 100, 2)) + '%'

'Average downgraded submission ratio: 14.43%'

In [18]:
df4 = pd.DataFrame(df[df.invalid == 1].groupby("date")["invalid"].count()).reset_index()
df4 = pd.merge(df4, df[df["disagree with severity"] == 1].groupby("date")["disagree with severity"].count(), on='date')
df4 = pd.merge(df4, df[df["duplicate"] == 1].groupby("date")["duplicate"].count(), on='date')
df4['date'] = pd.to_datetime(df4['date'])
df4

,date,invalid,disagree with severity,duplicate
0,2021-04-01,47,100,201
1,2021-05-01,26,31,67
2,2021-06-01,43,77,173
3,2021-07-01,110,84,228
4,2021-08-01,54,68,97
5,2021-09-01,218,93,371
6,2021-10-01,153,74,245
7,2021-11-01,393,118,849
8,2021-12-01,297,145,556
9,2022-01-01,870,189,1243


In [19]:
alt.Chart(df4, width=850, height=350, title="Tags presence over time").transform_fold(
    ["invalid", "disagree with severity", "duplicate"]
).mark_line(
    point=True
).encode(
    x=alt.X('date:T', title=""),
    y=alt.Y('value:Q', title=""),
    color=alt.Color('key:N', legend=alt.Legend(orient="top", labelFontSize=14, labelLimit=250)),
    tooltip=['date:T', 'value:Q']
)

/opt/hostedtoolcache/Python/3.11.1/x64/lib/python3.11/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)